# Import libraries

In [27]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd

# Get Cache

# Get Urls

In [28]:
def get_page_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)

    driver.find_element(By.XPATH,'//*[@id="uc-btn-accept-banner"]').click()



    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_links = soup.select('li.search-results__item a')

    url_list = []
    for element in s_links:
        link = element["href"]
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be/' not in link:
            link = 'https://www.immoweb.be/' + link
            url_list.append(link)
        else:
            url_list.append(link)
    return url_list



def main():
    url_list = []
    for x in range(1,3):
        urls = get_page_links(f'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=%7Bx%7D&orderBy=relevance%27')
        for url in urls:
            if url not in url_list:
                url_list.append(url)
    print(len(url_list))
    return url_list

print(main())

60
['https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-lambert/1200/10060574?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=6338459f18c32', 'https://www.immoweb.be/en/classified/new-rea

# Get Sessions

In [29]:
session = requests.Session()

# Get Elements_properties_raw

In [30]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text

    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [31]:
main_dictionary = dict()
for url in url_list:
    main_dictionary[url] = get_property(url)

# Get Elements_properties_list

In [32]:
data_main_list = list()
for index,element in enumerate(main_dictionary.values()):
    element_data = {
    'url': list(main_dictionary.keys())[index],
    'id': element['id'],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    df = pd.DataFrame(list1)
    df.to_csv("df_immo.csv")
    data_main_list.append(element_data)
print(df.head())

                                                 url        id Locality  \
0  https://www.immoweb.be/en/classified/house/for...  10008022     1050   
1      https://www.immoweb.be/en/classified/10125449  10125449     1070   

  Type of property Subtype of property      Price      Type of sale  \
0            HOUSE               HOUSE  1790000.0  residential_sale   
1  APARTMENT_GROUP     APARTMENT_GROUP        NaN        group_sale   

   Number of rooms  Area Fully equipped kitchen  ...  Open fire  Terrace  \
0              5.0  None     USA_HYPER_EQUIPPED  ...      False     True   
1              NaN  None                   None  ...      False     None   

  Terrace area Garden Garden area  Surface of the land  \
0         None   True       140.0                290.0   
1         None   None         NaN                  NaN   

   Surface area of the plot of land  Number of facades   Swimming pool  \
0                             290.0                   2          False   
1       

In [33]:
print (data_main_list)

[{'url': 'https://www.immoweb.be/en/classified/house/for-sale/ixelles/1050/10008022?searchId=6336981899ac0', 'id': 10008022, 'Locality': '1050', 'Type of property': 'HOUSE', 'Subtype of property': 'HOUSE', 'Price': 1790000, 'Type of sale': 'residential_sale', 'Number of rooms': 5, 'Area': None, 'Fully equipped kitchen': 'USA_HYPER_EQUIPPED', 'Furnished': False, 'Open fire': False, 'Terrace': True, 'Terrace area': None, 'Garden': True, 'Garden area': 140, 'Surface of the land': 290, 'Surface area of the plot of land': 290, 'Number of facades ': 2, 'Swimming pool': False, 'State of the building': 'AS_NEW'}, {'url': 'https://www.immoweb.be/en/classified/10125449', 'id': 10125449, 'Locality': '1070', 'Type of property': 'APARTMENT_GROUP', 'Subtype of property': 'APARTMENT_GROUP', 'Price': None, 'Type of sale': 'group_sale', 'Number of rooms': None, 'Area': None, 'Fully equipped kitchen': None, 'Furnished': False, 'Open fire': False, 'Terrace': None, 'Terrace area': None, 'Garden': None, 'G

# Save information into CSV

In [56]:
import csv

def main(list_dict):
    """a function that prints a csv starting from a dictionarires list 
        main(list_dict) """
    df = pd.DataFrame([list_dict[0],list_dict[1]])
    # we need at least 2 dict to create a csv file, so I made one with "list1" (a list with the first 2 links that I have)
    df.to_csv("df_immo.csv", index=False)
    for i in  range(2, len(data_main_list)): #
        #open csv in append mode ("a")
        file = open('df_immo.csv', "a")
        #use csv library to add a row to the csv file. The row has to be in a list format, so I've used the .values() method of the ditionary class
        csv.writer(file).writerow(data_main_list[i].values())
        print(i)
    #at the end close the csv file
    file.close

main(data_main_list)




2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
